# Module 12 Challenge
## Deliverable 2: Scrape and Analyze Mars Weather Data

In [ ]:
# Import relevant libraries
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from selenium.webdriver.chrome.service import Service
import os 

In [ ]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

### Step 1: Visit the Website

Use automated browsing to visit the [Mars Temperature Data Site](https://static.bc-edx.com/data/web/mars_facts/temperature.html). Inspect the page to identify which elements to scrape.

   > **Hint** To identify which elements to scrape, you might want to inspect the page by using Chrome DevTools to discover whether the table contains usable classes.


In [ ]:
# Visit the website
# https://static.bc-edx.com/data/web/mars_facts/temperature.html
url = "https://static.bc-edx.com/data/web/mars_facts/temperature.html"
browser.visit(url)

### Step 2: Scrape the Table

Create a Beautiful Soup object and use it to scrape the data in the HTML table.

Note that this can also be achieved by using the Pandas `read_html` function. However, use Beautiful Soup here to continue sharpening your web scraping skills.

In [ ]:
# Create a Beautiful Soup Object
html = browser.html
mars_soup = soup(html, 'html.parser')

In [ ]:
# Extract all rows of data
tabs_data = mars_soup.find_all(class_='data-row')
tabs_data

### Step 3: Store the Data

Assemble the scraped data into a Pandas DataFrame. The columns should have the same headings as the table on the website. Here’s an explanation of the column headings:

* `id`: the identification number of a single transmission from the Curiosity rover
* `terrestrial_date`: the date on Earth
* `sol`: the number of elapsed sols (Martian days) since Curiosity landed on Mars
* `ls`: the solar longitude
* `month`: the Martian month
* `min_temp`: the minimum temperature, in Celsius, of a single Martian day (sol)
* `pressure`: The atmospheric pressure at Curiosity's location

In [ ]:
# Create an empty list
empty_list = []
# Loop through the scraped data to create a list of rows
for row in tabs_data:
    data = row.find_all('td')
    empty_list.append([d.text.strip('[]') for d in data])


In [ ]:
# Create a Pandas DataFrame by using the list of rows and a list of the column names
column_names = ['id', 'terrestrial_date', 'sol', 'ls', 'month', 'min_temp', 'pressure']
mars_df = pd.DataFrame(empty_list, columns= column_names)

In [ ]:
# Confirm DataFrame was created successfully
mars_df

### Step 4: Prepare Data for Analysis

Examine the data types that are currently associated with each column. If necessary, cast (or convert) the data to the appropriate `datetime`, `int`, or `float` data types.

  > **Hint** You can use the Pandas `astype` and `to_datetime` methods to accomplish this task.


In [ ]:
# Examine data type of each column
mars_df.dtypes

In [ ]:
# Change data types for data analysis
mars_df['id']= mars_df['id'].astype(int)
mars_df['terrestrial_date']= pd.to_datetime(mars_df['terrestrial_date'], format= '%Y-%m-%d')
mars_df['sol']= mars_df['sol'].astype(int)
mars_df['ls']= mars_df['ls'].astype(int)
mars_df['month']= mars_df['month'].astype(int)
mars_df['min_temp']= mars_df['min_temp'].astype(float)
mars_df['pressure']= mars_df['pressure'].astype(float)

In [ ]:
# Confirm type changes were successful by examining data types again
mars_df.dtypes

### Step 5: Analyze the Data

Analyze your dataset by using Pandas functions to answer the following questions:

1. How many months exist on Mars?
2. How many Martian (and not Earth) days worth of data exist in the scraped dataset?
3. What are the coldest and the warmest months on Mars (at the location of Curiosity)? To answer this question:
    * Find the average the minimum daily temperature for all of the months.
    * Plot the results as a bar chart.
4. Which months have the lowest and the highest atmospheric pressure on Mars? To answer this question:
    * Find the average the daily atmospheric pressure of all the months.
    * Plot the results as a bar chart.
5. About how many terrestrial (Earth) days exist in a Martian year? To answer this question:
    * Consider how many days elapse on Earth in the time that Mars circles the Sun once.
    * Visually estimate the result by plotting the daily minimum temperature.


In [ ]:
# 1. How many months are there on Mars?
months = mars_df['month'].nunique()

print(f"There were {months} months on Mars.")

In [ ]:
# 2. How many Martian days' worth of data are there?
martian_days = mars_df['sol'].nunique()

print(f"There are {martian_days} Martian days' of data")

In [ ]:
# 3. What is the average low temperature by month?
avg_low_month_temp = mars_df['min_temp'].groupby(mars_df['month']).mean()

avg_low_month_temp

In [ ]:
# Plot the average temperature by month
plt.bar(avg_low_month_temp, avg_low_month_temp)
plt.xlabel('Months')
plt.ylabel('Temperature (in C°)')
plt.title('Average Low Temperature per Month')

plt.show()

In [ ]:
# Identify the coldest and hottest months in Curiosity's location
cold_month = mars_df['min_temp'].groupby(mars_df['month']).max().idxmin()

print(f"Month '[ {cold_month} ']' was the coldest month")

In [ ]:
hot_month = mars_df['min_temp'].groupby(mars_df['month']).max().idxmax()

print(f"Month '[' {hot_month} ']' was the hottest month.")

In [ ]:
# 4. Average pressure by Martian month
avg_pressure = mars_df['pressure'].groupby(mars_df['month']).mean()

print(f"The average pressure is: {avg_pressure}")

In [ ]:
# Plot the average pressure by month
plt.bar(avg_pressure, avg_pressure)
plt.xlabel('Months')
plt.ylabel('Atmopheric Pressure')
plt.title('Average Pressure on Mars')

plt.show()

In [ ]:
# 5. How many terrestrial (earth) days are there in a Martian year?
day_1 = mars_df['ls'].loc[0]

values = mars_df.loc[mars_df['ls'] == day_1]

one = values['terrestrial_date'].iloc[0]
three = values['terrestrial_date'].iloc[2]

earth_days = three - one

earth_days

On average, the third month has the coldest minimum temperature on Mars, and the eighth month is the warmest. But it is always very cold there in human terms!



Atmospheric pressure is, on average, lowest in the sixth month and highest in the ninth.

The distance from peak to peak is roughly 1425-750, or 675 days. A year on Mars appears to be about 675 days from the plot. Internet search confirms that a Mars year is equivalent to 687 earth days.

### Step 6: Save the Data

Export the DataFrame to a CSV file.

In [ ]:
# Write the data to a CSV
mars_df.to_csv('Mars.csv')

In [ ]:
browser.quit()